In [13]:
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn import preprocessing

Function to read the images and convert them to 5D vectors for each pixel:

In [9]:
def readImg(imgPath):
    img = Image.open(imgPath)
    pixels = img.convert('RGB')
    colours = np.array(pixels.getdata()).reshape(img.size + (3,))
    indicesArray = np.moveaxis(np.indices(img.size), 0, 2)
    allArray = np.dstack((indicesArray, colours)).reshape((-1, 5))

    df = pd.DataFrame(allArray, columns = ['y', 'x', 'r', 'g', 'b'])
    df = df[['x', 'y', 'r', 'g', 'b']]  # reorder columns
    return df

In [10]:
images = [os.getcwd() + '/Images/42049_colorBird.jpg', os.getcwd() + '/Images/3096_colorPlane.jpg']  # images are in a dir called Images
bird = readImg(images[0])
plane = readImg(images[1])
print(bird.shape, plane.shape)

(154401, 5) (154401, 5)


In [15]:
bird.head()

,x,y,r,g,b
0,0,0,59,87,111
1,1,0,60,88,112
2,2,0,61,89,113
3,3,0,63,91,115
4,4,0,65,93,117


In [14]:
bird.head().values

array([[  0,   0,  59,  87, 111],
       [  1,   0,  60,  88, 112],
       [  2,   0,  61,  89, 113],
       [  3,   0,  63,  91, 115],
       [  4,   0,  65,  93, 117]])

In [17]:
vals = bird.values
scaler = preprocessing.MinMaxScaler()
scaled = scaler.fit_transform(vals)
df = pd.DataFrame(scaled, columns = bird.columns)
df

,x,y,r,g,b
0,0.000000,0.0,0.241803,0.345238,0.449393
1,0.003125,0.0,0.245902,0.349206,0.453441
2,0.006250,0.0,0.250000,0.353175,0.457490
3,0.009375,0.0,0.258197,0.361111,0.465587
4,0.012500,0.0,0.266393,0.369048,0.473684
...,...,...,...,...,...
154396,0.987500,1.0,0.176230,0.218254,0.222672
154397,0.990625,1.0,0.184426,0.198413,0.214575
154398,0.993750,1.0,0.110656,0.111111,0.129555
154399,0.996875,1.0,0.106557,0.115079,0.153846
